In [1]:
import sys
import os

caminho_relativo = os.path.abspath('')
caminho_relativo = os.path.dirname(caminho_relativo)
sys.path.append(caminho_relativo)
ordem_das_etapas = ['static', 'insercao_datamart', 'extracao_site', 'venv']
for etapa in ordem_das_etapas:
    sys.path.append(os.path.join(caminho_relativo, etapa))

In [2]:
import datetime

In [4]:
from static.registrar_consultar import registers
inst_register = registers()

In [3]:

import pandas

inst_register = registers()

fichas = inst_register.consultar_notas('select * from monitoria.fichas')
colunas = inst_register.consultar_notas('select * from monitoria.colunas')
registros = inst_register.consultar_notas('select * from monitoria.registros')
registro_coluna = inst_register.consultar_notas('select * from monitoria.registro_coluna')
fichas = pandas.DataFrame(fichas)
colunas = pandas.DataFrame(colunas)
registros = pandas.DataFrame(registros)
registro_coluna = pandas.DataFrame(registro_coluna)

In [4]:
caminho_fichas_excel = r'\\55aspdcarq01\55atende\Administrativo\06 - Gerência Contact Center\03 - Call Center Sao Luis\02 - Monitoria de Qualidade\10.BASES'
data_atual = datetime.date.today() - datetime.timedelta(days=1)
lista_de_arquivos = os.listdir(caminho_fichas_excel)
lista_de_arquivos = [os.path.join(caminho_fichas_excel, linha) for linha in lista_de_arquivos if linha.endswith('.xls') and linha.startswith('{:02d}-{:04d}'.format(data_atual.month,data_atual.year))]


In [5]:
for ficha in lista_de_arquivos:
    split_ficha = ficha.split('\\')
    ficha = split_ficha[len(split_ficha) - 1]
    ficha = ficha.replace('.xls', '')
    ficha = ficha.replace('{:02d}-{:04d}'.format(data_atual.month,data_atual.year), '')
    ficha = ficha.strip()
    resultado_busca = fichas.loc[fichas['nome_ficha'] == ficha]
    if resultado_busca.empty:
        inst_register.registro_sucesso(
            dicionario= {
                'nome_ficha': ficha
            }, tabela='monitoria.fichas'
            )

In [6]:
ficha_fk = 2

In [7]:
# for ficha in lista_de_arquivos:
ficha_dataframe = pandas.read_html(lista_de_arquivos[0], keep_default_na=False, decimal=',', thousands='.')
if len(ficha_dataframe) != 1:
    raise AssertionError('Pandas leu o excel mas o tamanho de planilha foi diferente de 1')

ficha_dataframe = ficha_dataframe[0]

colunas_excel = ficha_dataframe.columns.values.tolist()

for coluna in colunas_excel:
    resultado_busca = colunas.loc[(colunas['ficha_fk'] == ficha_fk) & (colunas['nome_coluna'] == coluna)]
    if resultado_busca.empty:
        inst_register.registro_sucesso(
        dicionario= {
            'nome_coluna': coluna,
            'ficha_fk': ficha_fk
        }, tabela='monitoria.colunas'
        )

In [8]:
# for ficha in lista_de_arquivos:

df_somente_num_monitoria = ficha_dataframe['NUM_MONITORIA']

for valor_num_monitoria in df_somente_num_monitoria.values.tolist():
    resultado_busca = registros.loc[(registros['ficha_fk'] == ficha_fk) & (registros['num_monitoria'] == valor_num_monitoria)]
    if resultado_busca.empty:
        inst_register.registro_sucesso(
        dicionario= {
            'num_monitoria': valor_num_monitoria,
            'ficha_fk': ficha_fk
        }, tabela='monitoria.registros'
        )

In [9]:
dataframe_sem_num_monitoria = ficha_dataframe.drop('NUM_MONITORIA', axis=1)

In [10]:
colunas = inst_register.consultar_notas('select * from monitoria.colunas')
registros = inst_register.consultar_notas('select * from monitoria.registros')
registro_coluna = inst_register.consultar_notas('select * from monitoria.registro_coluna')

In [11]:
colunas = pandas.DataFrame(colunas)
registros = pandas.DataFrame(registros)
registro_coluna = pandas.DataFrame(registro_coluna)

In [12]:
1/0

ZeroDivisionError: division by zero

In [15]:
for nome_coluna, df_registro_coluna in dataframe_sem_num_monitoria.items():
    id_coluna = colunas.loc[(colunas['nome_coluna'] == nome_coluna) & (colunas['ficha_fk'] == ficha_fk)]
    id_coluna = id_coluna['id'].values.tolist()[0]
    for index_linha, coluna_loop in enumerate(df_registro_coluna.values.tolist()):
        id_registro = ficha_dataframe.loc[index_linha]['id']

KeyError: 'id'